In [1]:
!pip install sentence_transformers
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
working_folder = '/content/drive/My Drive/TransformersCode/03-restaurant/chatbot/'

In [4]:
import os
import pandas as pd

csv_file_path = os.path.join(working_folder, 'indian_food.csv')
df = pd.read_csv(csv_file_path)

df.head()

,name,ARname,ingredients,ARingredients,diet,ARdiet,prep_time,cook_time,flavor_profile,course,ARflavor_profile,ARcourse,state,region
0,Balu shahi,بالو شاهي,"Maida flour, yogurt, oil, sugar",دقيق مائدة، زبادي، زيت، سكر,vegetarian,نباتي,45,25,sweet,dessert,حلو,تحلية,West Bengal,East
1,Boondi,بوندي,"Gram flour, ghee, sugar",جرام دقيق، سمن، سكر,vegetarian,نباتي,80,30,sweet,dessert,حلو,تحلية,Rajasthan,West
2,Gajar ka halwa,جاجار كا حلوة,"Carrots, milk, sugar, ghee, cashews, raisins",جزر، حليب، سكر، سمن، كاجو، زبيب,vegetarian,نباتي,15,60,sweet,dessert,حلو,تحلية,Punjab,North
3,Ghevar,غيفار,"Flour, ghee, kewra, milk, clarified butter, su...",طحين، سمن، كيورا، حليب، زبدة، سكر، لوز، فستق، ...,vegetarian,نباتي,15,30,sweet,dessert,حلو,تحلية,Rajasthan,West
4,Gulab beef,جلاب بقر,"Milk powder, plain flour, baking powder, ghee,...",حليب بودرة ، دقيق عادي ، بيكنج باودر ، سمن ، ح...,vegetarian,نباتي,15,40,sweet,dessert,حلو,تحلية,West Bengal,East


In [5]:
QA_file_path = working_folder + 'AR_QA.csv'

QAfaiss_file_path = working_folder + 'QA_faiss_index.bin'

In [6]:
def create_qa_pairs(row):
    qa_pairs = []

    name = row['ARname']

    ingredients = row['ARingredients']

    diet = row['ARdiet']

    prep_time = row['prep_time']

    cook_time = row['cook_time']

    state = row['state']

    region = row['region']

    flavor_profile = row['ARflavor_profile']

    course = row['ARcourse']

    qa_pairs.append({"question": f"ما هي المكونات اللازمة لتحضير {name}؟", "answer": ingredients})
    qa_pairs.append({"question": f"ما هي المكونات المطلوبة لعمل {name}؟", "answer": ingredients})
    qa_pairs.append({"question": f"مما يتكون {name}؟", "answer": ingredients})

    qa_pairs.append({"question": f"ما هو نوع الحمية الغذائية ل {name}؟", "answer": diet})
    qa_pairs.append({"question": f"هل {name} نباتي أم غير نباتي؟", "answer": diet})
    qa_pairs.append({"question": f"ما هو النظام الغذائي ل {name}؟", "answer": diet})

    qa_pairs.append({"question": f"كم من الوقت يستغرق تحضير {name}؟", "answer": f"{prep_time} دقيقة"})
    qa_pairs.append({"question": f"ما هي مدة تحضير {name}؟", "answer": f"{prep_time} دقيقة"})
    qa_pairs.append({"question": f"كم دقيقة يستغرق تحضير {name}؟", "answer": f"{prep_time} دقيقة"})

    qa_pairs.append({"question": f"كم من الوقت يستغرق طهي {name}؟", "answer": f"{cook_time} دقيقة"})
    qa_pairs.append({"question": f"ما هي مدة طهي {name}؟", "answer": f"{cook_time} دقيقة"})
    qa_pairs.append({"question": f"كم دقيقة يستغرق طهي {name}؟", "answer": f"{cook_time} دقيقة"})

    qa_pairs.append({"question": f"ما هو أصل {name}؟", "answer": state})
    qa_pairs.append({"question": f"في أي ولاية تم اختراع {name}؟", "answer": state})
    qa_pairs.append({"question": f"إلى أي ولاية ينتمي {name}؟", "answer": state})

    qa_pairs.append({"question": f"في أي منطقة يتم تحضير {name}؟", "answer": region})
    qa_pairs.append({"question": f"ما هي المنطقة التي ينتمي إليها {name}؟", "answer": region})
    qa_pairs.append({"question": f"في أي منطقة تشتهر {name}؟", "answer": region})

    qa_pairs.append({"question": f"ما هو المذاق الخاص ب {name}؟", "answer": flavor_profile})
    qa_pairs.append({"question": f"كيف يكون طعم {name}؟", "answer": flavor_profile})
    qa_pairs.append({"question": f"ما هي نكهة {name}؟", "answer": flavor_profile})

    qa_pairs.append({"question": f"ما هو التصنيف الغذائي ل {name}؟", "answer": course})
    qa_pairs.append({"question": f"في أي وجبة يمكن تناول {name}؟", "answer": course})
    qa_pairs.append({"question": f"هل {name} يعد طبقاً رئيسياً أم جانبياً؟", "answer": course})

    return qa_pairs

In [7]:
qa_list = df.apply(create_qa_pairs, axis=1).sum()

qa_df = pd.DataFrame(qa_list)

qa_df.to_csv(QA_file_path, index=False, encoding='UTF-8')

print("Q&A pairs CSV file generated successfully.")

Q&A pairs CSV file generated successfully.


In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

def get_embeddings(texts):
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings.numpy()

questions = qa_df['question'].tolist()
question_embeddings = get_embeddings(questions)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense%2Fconfig.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [9]:
import faiss

index = faiss.IndexFlatIP(question_embeddings.shape[1])

faiss.normalize_L2(question_embeddings)

index.add(question_embeddings)

faiss.write_index(index, QAfaiss_file_path)